# Preliminaries

In [ ]:
# import modules
import numpy as np
... # your code here

In [54]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [86]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        #print(data)
        #print(m)
        #print(M)
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1
        
        #print(valid_features, D_try)
        #print(invalid_features)
        #print(valid_features[np.random.choice(len(valid_features), size=D_try, replace=False)])
        
        
        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            #print(n, n_min)
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                if len(valid_features) < D_try:
                    print(">")
                selected_indices = valid_features[np.random.choice(len(valid_features), size=D_try, replace=False)]
                children = make_density_split_node(node, N, selected_indices)
                stack.extend(children)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        if (x >= self.box[0]).all() and (x <= self.box[1]).all():
            #print(leaf.__dict__)
            return leaf.response * self.prior
        return 0

In [103]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
#... # your code here

import numpy as np
from matplotlib import pyplot as plt
from sklearn import model_selection

from sklearn.datasets import load_digits
digits = load_digits()
print(digits.keys())

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

trees = []
for i in range(10):
    density_tree = DensityTree()
    data_digit = data[target == i]
    #print(data_digit.shape)
    density_tree.train(data_digit, data_digit.shape[0] / len(target))
    trees.append(density_tree)
    
    #print(density_tree.predict(data[2]))



dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])
16.0
7.5
14.0
1.5
16.0
2.5
16.0
2.5
16.0
8.5
16.0
10.0
16.0
11.5
16.0
10.5
11.0
2.5
16.0
1.5
16.0
1.5
10.0
1.5
12.0
1.5
15.0
2.0
7.0
1.5
11.0
1.5
16.0
4.5
16.0
1.5
16.0
7.5
10.0
1.5
16.0
8.5
16.0
8.5
14.0
1.5


C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\nwolf\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [102]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box
    
    V = np.prod(M - m)
    #print(V)
    
    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = np.convolve(data_unique, np.ones(2), mode='valid')/2
                
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Compute the error
            
            mask = data_unique < t
            # left:
            left = data_unique[mask]
            V_left = V / (M[j] - m[j]) * (t - m[j])
            N_left = len(left)
            loo_error_left = N_left/(N*V_left) * (N_left/N - 2*((N_left - 1)/(N - 1)))
            
            right = data_unique[~mask]
            V_right = V / (M[j] - m[j]) * (M[j] - t)
            N_right = len(right)
            loo_error_right = N_right/(N*V_right) * (N_right/N - 2*((N_right - 1)/(N - 1)))
            
            loo_error = loo_error_left + loo_error_right
            #print(loo_error)
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    mask = node.data[:, j_min] < t_min
    
    left.data = node.data[mask] # store data in left node -- for subsequent splits
    left.box = m, M # store bounding box in left node
    left.box[1][j_min] = t_min
    
    
    right.data = node.data[~mask]
    right.box = m, M
    right.box[0][j_min] = t_min

    #print(j_min, t_min)
    
    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

In [73]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    v = np.prod(node.box[1] - node.box[0])
    node.response = n/(N*v)

# Decision Tree

In [ ]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                ... # your code here
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                ... # your code here
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return ... # your code here

In [22]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    ... # your code here

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = ... # data in left node
    left.labels = ... # corresponding labels
    right.data = ...
    right.labels = ...

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = ...
    node.threshold = ...

    # return the children (to be placed on the stack)
    return left, right    

In [32]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = ...
    node.response = ... # your code here

In [25]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return ... # your code here

# Evaluation of Density and Decision Tree

In [ ]:
# read and prepare the digits data
... # your code here

In [ ]:
# train trees, plot training error confusion matrices, and comment on your results
... # your code here

# Density and Decision Forest

In [7]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [35]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [6]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
#... # your code here

import numpy as np
from matplotlib import pyplot as plt
from sklearn import model_selection

from sklearn.datasets import load_digits
digits = load_digits()
print(digits.keys())

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]


d = DensityTree()
ones = data[target==1]
prior_one = ones.shape[0] / len(target)
d.train(data[target==1], prior_one)

dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])
